In [1]:
import numpy as np
import pandas as pd
import os
import glob
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn import metrics
%matplotlib inline
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
from mpl_toolkits.mplot3d import Axes3D
from sklearn.neighbors import NearestNeighbors
from itertools import combinations
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle
from sklearn.model_selection import cross_validate, cross_val_predict
from math import sqrt
from scipy import stats

# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

#load data 
df_list = []
path = r'F:\OneDrive - Nanyang Technological University\PhD Study\P10_TBM positional optimization\Original data'
allFiles = glob.glob(path + "/*.csv")
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df)
frame = pd.concat(list_)

In [2]:
TBMParameters = pd.DataFrame(frame, columns =['[19]Thrust Force','[20]Penetration',
                                           '[25]CHD Torque','[26]CHD RPM','[68]Earth Pressure(LM)','[71]Earth Pressure(RM)',
                                           '[262]Thrust Cylinders Right (Group A)','[263]Thrust Cylinders Down(Group B)',
                                           '[264]Thrust Cylinders Left (Group C)','[265]Thrust Cylinders Top(Group D)',
                                           '[21]Articulation Displacement A','[22]Articulation Displacement B',
                                           '[23]Articulation Displacement C','[24]Articulation Displacement D',
                                              '[85]H Deviation (Art)','[86]H Deviation (Tail)',
                                           '[88]V Deviation (Art)','[89]V Deviation (Tail)'])
                                             
TBMParameters = TBMParameters.apply(pd.to_numeric, errors='coerce')
#Remove nan
TBMParameters = TBMParameters.dropna()
TBMParameters = TBMParameters.loc[(TBMParameters!=0).all(1)]
TBMParameters = TBMParameters[(np.abs(stats.zscore(TBMParameters)) < 3).all(axis=1)]

reframe = series_to_supervised(TBMParameters,2,1)

In [10]:
TBMParameters.describe()

,[19]Thrust Force,[20]Penetration,[25]CHD Torque,[26]CHD RPM,[68]Earth Pressure(LM),[71]Earth Pressure(RM),[262]Thrust Cylinders Right (Group A),[263]Thrust Cylinders Down(Group B),[264]Thrust Cylinders Left (Group C),[265]Thrust Cylinders Top(Group D),[21]Articulation Displacement A,[22]Articulation Displacement B,[23]Articulation Displacement C,[24]Articulation Displacement D,[85]H Deviation (Art),[86]H Deviation (Tail),[88]V Deviation (Art),[89]V Deviation (Tail)
count,12233.000000,12233.000000,12233.000000,12233.000000,12233.000000,12233.000000,12233.000000,12233.000000,12233.000000,12233.000000,12233.000000,12233.000000,12233.000000,12233.000000,12233.000000,12233.000000,12233.000000,12233.000000
mean,23243.586283,30.533639,829.904766,1.234790,3.813935,3.786581,3939.605902,7645.354942,2656.967138,1524.357966,61.293877,57.299027,46.642606,42.604921,-12.274127,-25.076253,-1.815720,-25.492071
std,1377.478128,6.294524,164.147718,0.060043,0.214626,0.211805,588.445879,763.788960,667.317053,490.269378,7.887154,4.420008,5.265448,5.061247,10.967461,12.282607,9.666711,11.254859
min,13930.000000,7.000000,347.000000,0.950000,3.100000,3.090000,2201.000000,5218.000000,1010.000000,533.000000,38.000000,45.000000,39.000000,35.000000,-38.700000,-65.000000,-30.700000,-57.800000
25%,22591.000000,27.000000,717.000000,1.200000,3.680000,3.650000,3462.000000,7130.000000,2101.000000,1146.000000,58.000000,55.000000,43.000000,39.000000,-18.600000,-32.600000,-7.200000,-32.800000
50%,23311.000000,31.000000,806.000000,1.230000,3.820000,3.800000,3956.000000,7772.000000,2714.000000,1447.000000,61.000000,58.000000,45.000000,40.000000,-13.200000,-22.300000,-1.100000,-25.500000
75%,24057.000000,35.000000,916.000000,1.260000,3.960000,3.930000,4421.000000,8204.000000,3122.000000,1817.000000,66.000000,60.000000,50.000000,46.000000,-4.900000,-17.500000,4.900000,-18.400000
max,27729.000000,52.000000,1554.000000,1.510000,4.500000,4.420000,5741.000000,9698.000000,4660.000000,3212.000000,78.000000,67.000000,61.000000,58.000000,16.200000,13.800000,18.700000,-0.300000


In [6]:
#BPNN HDA
from sklearn.neural_network import MLPRegressor

X = reframe.iloc[:,:-4]
y = reframe.iloc[:,-4]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

mlp = MLPRegressor(hidden_layer_sizes=(8), solver='adam',max_iter=1000,random_state =1)
mlp.fit(X_train.iloc[:8000,:],y_train[:8000])
mlp_pred=mlp.predict(X_test)

mae = mean_absolute_error(y_test, mlp_pred)
rmse = sqrt(mean_squared_error(y_test, mlp_pred))
r2 = r2_score(y_test, mlp_pred)
print('Test MAE: %.3f' % mae)
print('Test RMSE: %.3f' % rmse)
print(r2)

np.savetxt('mlp_HDA.csv',mlp_pred,delimiter=',')

Test MAE: 1.204
Test RMSE: 1.480
0.9612002357787026


In [7]:
X = reframe.iloc[:,:-4]
y = reframe.iloc[:,-3]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

mlp = MLPRegressor(hidden_layer_sizes=(8), solver='adam',max_iter=1000,random_state =1)
mlp.fit(X_train.iloc[:8000,:],y_train[:8000])
mlp_pred=mlp.predict(X_test)

mae = mean_absolute_error(y_test, mlp_pred)
rmse = sqrt(mean_squared_error(y_test, mlp_pred))
r2 = r2_score(y_test, mlp_pred)
print('Test MAE: %.3f' % mae)
print('Test RMSE: %.3f' % rmse)
print(r2)

np.savetxt('mlp_HDT.csv',mlp_pred,delimiter=',')

Test MAE: 2.679
Test RMSE: 3.738
0.8582585665237459


In [8]:
X = reframe.iloc[:,:-4]
y = reframe.iloc[:,-2]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

mlp = MLPRegressor(hidden_layer_sizes=(8), solver='adam',max_iter=1000,random_state =1)
mlp.fit(X_train.iloc[:8000,:],y_train[:8000])
mlp_pred=mlp.predict(X_test)

mae = mean_absolute_error(y_test, mlp_pred)
rmse = sqrt(mean_squared_error(y_test, mlp_pred))
r2 = r2_score(y_test, mlp_pred)
print('Test MAE: %.3f' % mae)
print('Test RMSE: %.3f' % rmse)
print(r2)

np.savetxt('mlp_VDA.csv',mlp_pred,delimiter=',')

Test MAE: 1.316
Test RMSE: 1.457
0.965055083991841


In [9]:
X = reframe.iloc[:,:-4]
y = reframe.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

mlp = MLPRegressor(hidden_layer_sizes=(8), solver='adam',max_iter=1000,random_state =1)
mlp.fit(X_train.iloc[:8000,:],y_train[:8000])
mlp_pred=mlp.predict(X_test)

mae = mean_absolute_error(y_test, mlp_pred)
rmse = sqrt(mean_squared_error(y_test, mlp_pred))
r2 = r2_score(y_test, mlp_pred)
print('Test MAE: %.3f' % mae)
print('Test RMSE: %.3f' % rmse)
print(r2)

np.savetxt('mlp_VDT.csv',mlp_pred,delimiter=',')

Test MAE: 1.234
Test RMSE: 1.486
0.9859837553767258


In [6]:
import xgboost as xgb

X = reframe.iloc[:,:-4]
y = reframe.iloc[:,-4]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

xgb= xgb.XGBRegressor(n_estimators=60, max_depth = 15, learning_rate=0.15, random_state =1)
params={
    'n_estimators':[200, 400, 600, 800, 1000],
    'max_depth':[5, 10, 15],
    'learning_rate':[0.1, 0.15, 0.2]
}
gsearch2 = GridSearchCV(estimator=xgb, param_grid=params, scoring='neg_mean_squared_error', cv=5, verbose=2, n_jobs=5)
gsearch2.fit(X_train.iloc[:8000,:],y_train[:8000])
gsearch2.best_params_, gsearch2.best_score_

Fitting 5 folds for each of 45 candidates, totalling 225 fits


({'learning_rate': 0.15, 'max_depth': 15, 'n_estimators': 200},
 -2.5786436865875535)

In [16]:
#XGBoost HDA
import xgboost as xgb

X = reframe.iloc[:,:-4]
y = reframe.iloc[:,-4]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

xgb= xgb.XGBRegressor(n_estimators=200, max_depth = 15, learning_rate=0.15, random_state =1)
xgb.fit(X_train,y_train)
xgb_pred = xgb.predict(X_test)

mae = mean_absolute_error(y_test, xgb_pred)
rmse = sqrt(mean_squared_error(y_test, xgb_pred))
r2 = r2_score(y_test, xgb_pred)
print('Test MAE: %.3f' % mae)
print('Test RMSE: %.3f' % rmse)
print(r2)

np.savetxt('xgb_HDA.csv',xgb_pred,delimiter=',')

Test MAE: 0.806
Test RMSE: 1.208
0.9741641068022024


In [53]:
#XGBoost HDT
import xgboost as xgb
reframe = series_to_supervised(TBMParameters,2,1)
X = reframe.iloc[:,:-4]
y = reframe.iloc[:,-3]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

xgb= xgb.XGBRegressor(n_estimators=200, max_depth = 15, learning_rate=0.15, random_state =1)
xgb.fit(X_train.iloc[:8000,:],y_train[:8000])
xgb_pred = xgb.predict(X_test)

mae = mean_absolute_error(y_test, xgb_pred)
rmse = sqrt(mean_squared_error(y_test, xgb_pred))
r2 = r2_score(y_test, xgb_pred)
print('Test MAE: %.3f' % mae)
print('Test RMSE: %.3f' % rmse)
print(r2)

np.savetxt('xgb_HDT.csv',xgb_pred,delimiter=',')

Test MAE: 3.623
Test RMSE: 4.849
0.7614471105689025


In [23]:
#XGBoost VDA
import xgboost as xgb
X = reframe.iloc[:,:-4]
y = reframe.iloc[:,-2]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

xgb= xgb.XGBRegressor(n_estimators=200, max_depth = 15, learning_rate=0.15, random_state =1)
xgb.fit(X_train.iloc[:8000,:],y_train[:8000])
xgb_pred = xgb.predict(X_test)

mae = mean_absolute_error(y_test, xgb_pred)
rmse = sqrt(mean_squared_error(y_test, xgb_pred))
r2 = r2_score(y_test, xgb_pred)
print('Test MAE: %.3f' % mae)
print('Test RMSE: %.3f' % rmse)
print(r2)

np.savetxt('xgb_VDA.csv',xgb_pred,delimiter=',')

Test MAE: 0.679
Test RMSE: 1.061
0.981454279067023


In [24]:
#XGBoost VDT
import xgboost as xgb
X = reframe.iloc[:,:-4]
y = reframe.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

xgb= xgb.XGBRegressor(n_estimators=200, max_depth = 15, learning_rate=0.15, random_state =1)
xgb.fit(X_train.iloc[:8000,:],y_train[:8000])
xgb_pred = xgb.predict(X_test)

mae = mean_absolute_error(y_test, xgb_pred)
rmse = sqrt(mean_squared_error(y_test, xgb_pred))
r2 = r2_score(y_test, xgb_pred)
print('Test MAE: %.3f' % mae)
print('Test RMSE: %.3f' % rmse)
print(r2)

np.savetxt('xgb_VDT.csv',xgb_pred,delimiter=',')

Test MAE: 0.561
Test RMSE: 0.857
0.9953327085549065


In [27]:
#LSTM HDA
import tensorflow as tf
from tensorflow import keras

values = TBMParameters.values
values=values.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
reframe = series_to_supervised(scaled, 2,1)
reframed=reframe.values
train = reframed[:int(reframed.shape[0]*0.8), :]
test = reframed[int(reframed.shape[0]*0.8):, :]
train_X, train_y = train[:,:-4], train[:,-4]
test_X, test_y = test[:,:-4], test[:,-4]

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

model = keras.models.Sequential([
    keras.layers.LSTM(8, activation='elu', return_sequences=True, input_shape=(train_X.shape[1], train_X.shape[2])),
    keras.layers.Dense(1)
])

model.compile(loss = "mean_squared_error", optimizer = 'adam')

history = model.fit(train_X, train_y, epochs=100, batch_size=32, verbose=0, shuffle=False, validation_data=(test_X, test_y))

#Output result
y_predict = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
y_predict = y_predict.reshape(len(y_predict),1)

# invert scaling for forecast
inv_y_test = np.concatenate((test_X,y_predict,test[:,-3:]), axis=1)
inv_y_test = inv_y_test[:,-18:]
inv_y_test = scaler.inverse_transform(inv_y_test)
inv_y_predict=inv_y_test[:,-4]
 
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y_train = np.concatenate((test_X,test_y,test[:,-3:]), axis=1)
inv_y_train = inv_y_train[:,-18:]
inv_y_train = scaler.inverse_transform(inv_y_train)
inv_y = inv_y_train[:, -4]

mae = mean_absolute_error(inv_y, inv_y_predict)
rmse = sqrt(mean_squared_error(inv_y, inv_y_predict))
r2 = r2_score(inv_y, inv_y_predict)
print('Test MAE: %.3f' % mae)
print('Test RMSE: %.3f' % rmse)
print(r2)

LSTM_HDA = np.column_stack((inv_y_predict, inv_y))
np.savetxt('LSTM_HDA.csv',LSTM_HDA,delimiter=',')

Epoch 1/100
306/306 - 0s - loss: 0.0307 - val_loss: 0.0118
Epoch 2/100
306/306 - 0s - loss: 0.0170 - val_loss: 0.0100
Epoch 3/100
306/306 - 0s - loss: 0.0111 - val_loss: 0.0087
Epoch 4/100
306/306 - 0s - loss: 0.0082 - val_loss: 0.0081
Epoch 5/100
306/306 - 0s - loss: 0.0069 - val_loss: 0.0078
Epoch 6/100
306/306 - 0s - loss: 0.0062 - val_loss: 0.0074
Epoch 7/100
306/306 - 0s - loss: 0.0057 - val_loss: 0.0071
Epoch 8/100
306/306 - 0s - loss: 0.0053 - val_loss: 0.0067
Epoch 9/100
306/306 - 0s - loss: 0.0050 - val_loss: 0.0064
Epoch 10/100
306/306 - 0s - loss: 0.0047 - val_loss: 0.0061
Epoch 11/100
306/306 - 0s - loss: 0.0045 - val_loss: 0.0058
Epoch 12/100
306/306 - 0s - loss: 0.0042 - val_loss: 0.0055
Epoch 13/100
306/306 - 0s - loss: 0.0039 - val_loss: 0.0053
Epoch 14/100
306/306 - 0s - loss: 0.0036 - val_loss: 0.0052
Epoch 15/100
306/306 - 0s - loss: 0.0033 - val_loss: 0.0051
Epoch 16/100
306/306 - 0s - loss: 0.0030 - val_loss: 0.0050
Epoch 17/100
306/306 - 0s - loss: 0.0028 - val_lo

In [28]:
#LSTM HDT
train_X, train_y = train[:,:-4], train[:,-3]
test_X, test_y = test[:,:-4], test[:,-3]

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

model = keras.models.Sequential([
    keras.layers.LSTM(8, activation='elu', return_sequences=True, input_shape=(train_X.shape[1], train_X.shape[2])),
    keras.layers.Dense(1)
])

model.compile(loss = "mean_squared_error", optimizer = 'adam')

history = model.fit(train_X, train_y, epochs=100, batch_size=32, verbose=0, shuffle=False, validation_data=(test_X, test_y))

#Output result
y_predict = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
y_predict = y_predict.reshape(len(y_predict),1)

# invert scaling for forecast
inv_y_test = np.concatenate((test_X,test[:,-3:-2],y_predict,test[:,-2:]), axis=1)
inv_y_test = inv_y_test[:,-18:]
inv_y_test = scaler.inverse_transform(inv_y_test)
inv_y_predict=inv_y_test[:,-3]
 
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y_train = np.concatenate((test_X,test[:,-3:-2],test_y,test[:,-2:]), axis=1)
inv_y_train = inv_y_train[:,-18:]
inv_y_train = scaler.inverse_transform(inv_y_train)
inv_y = inv_y_train[:, -3]

mae = mean_absolute_error(inv_y, inv_y_predict)
rmse = sqrt(mean_squared_error(inv_y, inv_y_predict))
r2 = r2_score(inv_y, inv_y_predict)
print('Test MAE: %.3f' % mae)
print('Test RMSE: %.3f' % rmse)
print(r2)

LSTM_HDT = np.column_stack((inv_y_predict, inv_y))
np.savetxt('LSTM_HDT.csv',LSTM_HDT,delimiter=',')

Test MAE: 2.824
Test RMSE: 3.849
0.8497247433320041


In [29]:
#LSTM VDA
train_X, train_y = train[:,:-4], train[:,-2]
test_X, test_y = test[:,:-4], test[:,-2]

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

model = keras.models.Sequential([
    keras.layers.LSTM(8, activation='elu', return_sequences=True, input_shape=(train_X.shape[1], train_X.shape[2])),
    keras.layers.Dense(1)
])

model.compile(loss = "mean_squared_error", optimizer = 'adam')

history = model.fit(train_X, train_y, epochs=100, batch_size=32, verbose=0, shuffle=False, validation_data=(test_X, test_y))

#Output result
y_predict = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
y_predict = y_predict.reshape(len(y_predict),1)

# invert scaling for forecast
inv_y_test = np.concatenate((test_X,test[:,-3:-1],y_predict,test[:,-1:]), axis=1)
inv_y_test = inv_y_test[:,-18:]
inv_y_test = scaler.inverse_transform(inv_y_test)
inv_y_predict=inv_y_test[:,-2]
 
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y_train = np.concatenate((test_X,test[:,-3:-1],test_y,test[:,-1:]), axis=1)
inv_y_train = inv_y_train[:,-18:]
inv_y_train = scaler.inverse_transform(inv_y_train)
inv_y = inv_y_train[:, -2]

mae = mean_absolute_error(inv_y, inv_y_predict)
rmse = sqrt(mean_squared_error(inv_y, inv_y_predict))
r2 = r2_score(inv_y, inv_y_predict)
print('Test MAE: %.3f' % mae)
print('Test RMSE: %.3f' % rmse)
print(r2)

LSTM_VDA = np.column_stack((inv_y_predict, inv_y))
np.savetxt('LSTM_VDA.csv',LSTM_VDA,delimiter=',')

Test MAE: 1.262
Test RMSE: 1.636
0.9559475875596726


In [30]:
#LSTM VDT
train_X, train_y = train[:,:-4], train[:,-1]
test_X, test_y = test[:,:-4], test[:,-1]

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

model = keras.models.Sequential([
    keras.layers.LSTM(8, activation='elu', return_sequences=True, input_shape=(train_X.shape[1], train_X.shape[2])),
    keras.layers.Dense(1)
])

model.compile(loss = "mean_squared_error", optimizer = 'adam')

history = model.fit(train_X, train_y, epochs=100, batch_size=32, verbose=0, shuffle=False, validation_data=(test_X, test_y))

#Output result
y_predict = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
y_predict = y_predict.reshape(len(y_predict),1)

# invert scaling for forecast
inv_y_test = np.concatenate((test_X,test[:,-3:],y_predict), axis=1)
inv_y_test = inv_y_test[:,-18:]
inv_y_test = scaler.inverse_transform(inv_y_test)
inv_y_predict=inv_y_test[:,-1]
 
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y_train = np.concatenate((test_X,test[:,-3:],test_y), axis=1)
inv_y_train = inv_y_train[:,-18:]
inv_y_train = scaler.inverse_transform(inv_y_train)
inv_y = inv_y_train[:, -1]

mae = mean_absolute_error(inv_y, inv_y_predict)
rmse = sqrt(mean_squared_error(inv_y, inv_y_predict))
r2 = r2_score(inv_y, inv_y_predict)
print('Test MAE: %.3f' % mae)
print('Test RMSE: %.3f' % rmse)
print(r2)

LSTM_VDT = np.column_stack((inv_y_predict, inv_y))
np.savetxt('LSTM_VDT.csv',LSTM_VDT,delimiter=',')

Test MAE: 0.926
Test RMSE: 1.232
0.9903664040580574


In [36]:
reframe.shape

(12231, 54)

In [47]:
#PCA-GRU HDA
from sklearn.decomposition import PCA

reframed = reframe.values
inputs = reframed[:,:-4]
output = reframed[:,-4:]

#PCA
pca = PCA(n_components=50)
pca.fit(inputs)
pca_TBM = pca.transform(inputs)
pca_TBM = pca_TBM[:,:20]

reframed = np.column_stack((pca_TBM,output))
train = reframed[:int(reframed.shape[0]*0.8), :]
test = reframed[int(reframed.shape[0]*0.8):, :]
train_X, train_y = train[:,:-4], train[:,-4]
test_X, test_y = test[:,:-4], test[:,-4]

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

model = keras.models.Sequential([
    keras.layers.GRU(20, return_sequences=True, input_shape=(train_X.shape[1], train_X.shape[2])),
    keras.layers.Dense(8),
    keras.layers.Dense(1)
])

model.compile(loss = "mean_squared_error", optimizer = 'adam')

history = model.fit(train_X, train_y, epochs=50, batch_size=20, verbose=0, shuffle=False, validation_data=(test_X, test_y))

#Output result
y_predict = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
y_predict = y_predict.reshape(len(y_predict),1)

# invert scaling for forecast
inv_y_test = np.concatenate((inputs[int(reframed.shape[0]*0.8):, :],y_predict,test[:,-3:]), axis=1)
inv_y_test = inv_y_test[:,-18:]
inv_y_test = scaler.inverse_transform(inv_y_test)
inv_y_predict=inv_y_test[:,-4]
 
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y_train = np.concatenate((inputs[int(reframed.shape[0]*0.8):, :],test_y,test[:,-3:]), axis=1)
inv_y_train = inv_y_train[:,-18:]
inv_y_train = scaler.inverse_transform(inv_y_train)
inv_y = inv_y_train[:, -4]

mae = mean_absolute_error(inv_y, inv_y_predict)
rmse = sqrt(mean_squared_error(inv_y, inv_y_predict))
r2 = r2_score(inv_y, inv_y_predict)
print('Test MAE: %.3f' % mae)
print('Test RMSE: %.3f' % rmse)
print(r2)

GRU_HDA = np.column_stack((inv_y_predict, inv_y))
np.savetxt('GRU_HDA.csv',GRU_HDA,delimiter=',')

Test MAE: 1.814
Test RMSE: 2.323
0.9044683275320652


In [48]:
#PCA-GRU HDT

train_X, train_y = train[:,:-4], train[:,-3]
test_X, test_y = test[:,:-4], test[:,-3]

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

model = keras.models.Sequential([
    keras.layers.GRU(20, return_sequences=True, input_shape=(train_X.shape[1], train_X.shape[2])),
    keras.layers.Dense(8),
    keras.layers.Dense(1)
])

model.compile(loss = "mean_squared_error", optimizer = 'adam')

history = model.fit(train_X, train_y, epochs=50, batch_size=20, verbose=0, shuffle=False, validation_data=(test_X, test_y))

#Output result
y_predict = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
y_predict = y_predict.reshape(len(y_predict),1)

# invert scaling for forecast
inv_y_test = np.concatenate((inputs[int(reframed.shape[0]*0.8):, :],y_predict,test[:,-2:]), axis=1)
inv_y_test = inv_y_test[:,-18:]
inv_y_test = scaler.inverse_transform(inv_y_test)
inv_y_predict=inv_y_test[:,-3]
 
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y_train = np.concatenate((inputs[int(reframed.shape[0]*0.8):, :],test_y,test[:,-2:]), axis=1)
inv_y_train = inv_y_train[:,-18:]
inv_y_train = scaler.inverse_transform(inv_y_train)
inv_y = inv_y_train[:, -3]

mae = mean_absolute_error(inv_y, inv_y_predict)
rmse = sqrt(mean_squared_error(inv_y, inv_y_predict))
r2 = r2_score(inv_y, inv_y_predict)
print('Test MAE: %.3f' % mae)
print('Test RMSE: %.3f' % rmse)
print(r2)

GRU_HDT = np.column_stack((inv_y_predict, inv_y))
np.savetxt('GRU_HDT.csv',GRU_HDT,delimiter=',')

Test MAE: 2.972
Test RMSE: 4.373
0.8059988015431288


In [49]:
#PCA-GRU VDA

train_X, train_y = train[:,:-4], train[:,-2]
test_X, test_y = test[:,:-4], test[:,-2]

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

model = keras.models.Sequential([
    keras.layers.GRU(20, return_sequences=True, input_shape=(train_X.shape[1], train_X.shape[2])),
    keras.layers.Dense(8),
    keras.layers.Dense(1)
])

model.compile(loss = "mean_squared_error", optimizer = 'adam')

history = model.fit(train_X, train_y, epochs=50, batch_size=20, verbose=0, shuffle=False, validation_data=(test_X, test_y))

#Output result
y_predict = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
y_predict = y_predict.reshape(len(y_predict),1)

# invert scaling for forecast
inv_y_test = np.concatenate((inputs[int(reframed.shape[0]*0.8):, :],y_predict,test[:,-1:]), axis=1)
inv_y_test = inv_y_test[:,-18:]
inv_y_test = scaler.inverse_transform(inv_y_test)
inv_y_predict=inv_y_test[:,-2]
 
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y_train = np.concatenate((inputs[int(reframed.shape[0]*0.8):, :],test_y,test[:,-1:]), axis=1)
inv_y_train = inv_y_train[:,-18:]
inv_y_train = scaler.inverse_transform(inv_y_train)
inv_y = inv_y_train[:, -2]

mae = mean_absolute_error(inv_y, inv_y_predict)
rmse = sqrt(mean_squared_error(inv_y, inv_y_predict))
r2 = r2_score(inv_y, inv_y_predict)
print('Test MAE: %.3f' % mae)
print('Test RMSE: %.3f' % rmse)
print(r2)

GRU_VDA = np.column_stack((inv_y_predict, inv_y))
np.savetxt('GRU_VDA.csv',GRU_VDA,delimiter=',')

Test MAE: 2.120
Test RMSE: 2.637
0.885488499050487


In [50]:
#PCA-GRU VDT

train_X, train_y = train[:,:-4], train[:,-1]
test_X, test_y = test[:,:-4], test[:,-1]

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

model = keras.models.Sequential([
    keras.layers.GRU(20, return_sequences=True, input_shape=(train_X.shape[1], train_X.shape[2])),
    keras.layers.Dense(8),
    keras.layers.Dense(1)
])

model.compile(loss = "mean_squared_error", optimizer = 'adam')

history = model.fit(train_X, train_y, epochs=50, batch_size=20, verbose=0, shuffle=False, validation_data=(test_X, test_y))

#Output result
y_predict = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
y_predict = y_predict.reshape(len(y_predict),1)

# invert scaling for forecast
inv_y_test = np.concatenate((inputs[int(reframed.shape[0]*0.8):, :],y_predict), axis=1)
inv_y_test = inv_y_test[:,-18:]
inv_y_test = scaler.inverse_transform(inv_y_test)
inv_y_predict=inv_y_test[:,-1]
 
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y_train = np.concatenate((inputs[int(reframed.shape[0]*0.8):, :],test_y), axis=1)
inv_y_train = inv_y_train[:,-18:]
inv_y_train = scaler.inverse_transform(inv_y_train)
inv_y = inv_y_train[:, -1]

mae = mean_absolute_error(inv_y, inv_y_predict)
rmse = sqrt(mean_squared_error(inv_y, inv_y_predict))
r2 = r2_score(inv_y, inv_y_predict)
print('Test MAE: %.3f' % mae)
print('Test RMSE: %.3f' % rmse)
print(r2)

GRU_VDT = np.column_stack((inv_y_predict, inv_y))
np.savetxt('GRU_VDT.csv',GRU_VDT,delimiter=',')

Test MAE: 1.234
Test RMSE: 1.534
0.9850514436144666
